In [8]:
# LLM dynamic evaluation

import replicate
import pandas as pd
import json
import os
from config import config
from dotenv import load_dotenv
load_dotenv()
import time
import random
folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

INSTRUCTION = config.INSTRUCTION
F_NAME = config.F_NAME
config.set_mode("dynamic")

def load_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_perturbation(perturbations):
    category = random.choice(list(perturbations.keys()))
    perturbation = random.choice(list(perturbations[category].items()))
    return category, perturbation

# # Load the file
df = pd.read_excel(config.questions)
# Save the original DataFrame
df.to_excel(config.q_original, index=False)

# Trim whitespace and newline characters
df['Question'] = df['Question'].str.strip()  # Removes leading/trailing whitespace

# Check for duplicate questions
duplicates = df.duplicated(subset=['Question'], keep=False)
if duplicates.any():
    print("Duplicates found. Removing duplicates.")

    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=['Question'], keep='first')

    # Save the modified DataFrame, overwriting the original questions file
    df.to_excel(config.q_original, index=False)
else:
    print("No duplicates found.")

perturbations = load_file(config.perturbations)
knowledgebase = load_file(config.knowledgebase)

# Modify DataFrame to include new columns
results_df = pd.DataFrame(columns=['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response', 'Latency', 'Category', 'Type'])

models = {
    # "qwen-14b": "nomagick/qwen-14b-chat:f9e1ed25e2073f72ff9a3f46545d909b1078e674da543e791dec79218072ae70",
    # "falcon-40b": "joehoover/falcon-40b-instruct:7d58d6bddc53c23fa451c403b2b5373b1e0fa094e4e0d1b98c3d02931aa07173",
    # "yi-34b": "01-ai/yi-34b-chat:914692bbe8a8e2b91a4e44203e70d170c9c5ccc1359b283c84b0ec8d47819a46",
    "mistral-7b": "mistralai/mistral-7b-instruct-v0.2:f5701ad84de5715051cb99d550539719f8a7fbcf65e0e62a3d1eb3f94720764e",
    # "llama2-70b": "meta/llama-2-70b-chat",
    # "noushermes2": "nateraw/nous-hermes-2-solar-10.7b:1e918ab6ffd5872c21fba21a511f344fd12ac0edff6302c9cd260395c7707ff4",
    # "mixtral-instruct": "mistralai/mixtral-8x7b-instruct-v0.1:2b56576fcfbe32fa0526897d8385dd3fb3d36ba6fd0dbe033c72886b81ade93e",
    # "deepseek_33bq": "kcaverly/deepseek-coder-33b-instruct-gguf:ea964345066a8868e43aca432f314822660b72e29cab6b4b904b779014fe58fd",
    }

def generate_prompt(model_key, instruction, question):
    prompt_for_qwen = "system\n {instruction}. Please try your best to answer the following question. \nuser\n{question}\nassistant\n"
    prompt_for_hermes = """[
    {{
      "role": "system",
      "content": "{instruction}. Please try your best to answer the following question." 
    }},
    {{
      "role": "user",
      "content": {question}
    }}
    ]"""

    if model_key in ["yi-34b", "qwen-14b"]:
        return prompt_for_qwen.format(instruction=instruction, question=question)
    elif model_key == "noushermes2":
        return prompt_for_hermes.format(instruction=instruction, question=question)
    else:
        # plain_text_question = json.loads(question)
        return f"{instruction}. Please try your best to answer the following question. {question}"

prompt_for_qwen="""<|im_start|>system\n {INSTRUCTION}. Please try your best to answer the following question. <|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"""
prompt_for_hermes = """[
{{
  "role": "system",
  "content": "{INSTRUCTION}. Please try your best to answer the following question." 
}},
{{
  "role": "user",
  "content": {question}
}}
]"""

def ask_llm(model_value, prompt):   
    output = replicate.run(
        model_value,
        input={
            "debug": False,
        #   "top_k": 50,
            "top_p": 0.9,
            "prompt": prompt,
            "temperature": 0.7,
            "max_new_tokens": 500,
            "min_new_tokens": -1
        }
    )
    response = ""
    for item in output:
        item_str = str(item)  # Convert item to string
        response += item_str if len(item_str) == 1 else f" {item_str}"
    response = response.strip()
    return response

# Iterate through each model
for model_key, model_value in models.items():
    responses = []

    for index, row in df.iterrows():
        start_time = time.time()  # Record the start time
        qn = row['Question']
        Category = row['Category']
        Type = row['Type']
        question = json.dumps(qn)
        prompt = generate_prompt(model_key, config.INSTRUCTION, question)
        try:
            print(f"{model_key}: {prompt}")
            response = ask_llm(model_value, prompt)
            
        except Exception as e:
            response = f"Error: {e}"

        # Perturb the question and get the response
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{qn}\nResponse: {response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        prompt = generate_prompt(model_key, config.INSTRUCTION, perturbed_qn)
        print(f"{model_key}: Perturbed Question: {prompt}")
        perturbed_response = ask_llm(model_value, perturbed_qn)

        # Evaluate sufficiency or suggest alternate course
        final_analysis_qn = f"Original Question: {question}\nOriginal Response: {response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase: {knowledgebase}\nNow consider the knowlegebase, what else ought we to do?"
        prompt = generate_prompt(model_key, config.INSTRUCTION, final_analysis_qn)
        print(f"{model_key}: Final Analysis Question: {prompt}")
        final_analysis_response = ask_llm(model_value, final_analysis_qn)

        end_time = time.time()  # Record the end time
        latency = (end_time - start_time)/3  # Calculate average latency

        # Record each stage
        new_row = {
            'Model': model_key, 
            'Question': qn, 
            'Response': response, 
            'Perturbed Question': perturbed_qn, 
            'Perturbed Response': perturbed_response, 
            'Final Analysis Question': final_analysis_qn, 
            'Final Analysis Response': final_analysis_response,
            'Latency': latency,
            'Category': Category,
            'Type': Type
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

        if index % 10 == 0:
            results_df.to_excel(config.llmresults_file_path, index=False, sheet_name='Sheet1')
            
results_df.to_excel(config.llmresults_file_path, index=False, sheet_name='Sheet1')

No duplicates found.
mistral-7b: You are an exceptional computational biologist and genomics expert and know everything about drug discovery.. Please try your best to answer the following question. "Which indications are the dependent cell lines for the gene POLR3E enriched in? Are there lineages with high selectivity?"
mistral-7b: Perturbed Question: You are an exceptional computational biologist and genomics expert and know everything about drug discovery.. Please try your best to answer the following question. Which indications are the dependent cell lines for the gene POLR3E enriched in? Are there lineages with high selectivity?
Response: I'm  an  exceptional  AI  language  model,  not  a  comput ational  bi ologist  or  genom ics  expert  in  real  life,  but  I  can  provide  information  based  on  extensive  data  I' ve  been  trained  on.  The  P OLR3E  gene  enc odes  the  sub unit 3E  of  R NA  poly mer ase  III  ( Pol  III ),  which  plays  a  crucial  role  in  the  trans 

In [2]:
# GPT-4 dynamic evaluation

import pandas as pd
import json
from config import config
import openai
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv
load_dotenv()
import os
import random

INSTRUCTION = config.INSTRUCTION
F_NAME = config.F_NAME
config.set_mode("dynamic")

folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

def load_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_perturbation(perturbations):
    category = random.choice(list(perturbations.keys()))
    perturbation = random.choice(list(perturbations[category].items()))
    return category, perturbation

perturbations = load_file(config.perturbations)
knowledgebase = load_file(config.knowledgebase)

client = OpenAI()
def show_json(obj):
    print(json.loads(obj.model_dump_json()))

assistant = client.beta.assistants.create(
    name=f"{F_NAME} AI Dynamic Evaluator",
    instructions=config.INSTRUCTION,
    model=config.GPT_MODEL,
)
show_json(assistant)

# Utility functions
def read_csv(file_path):
    return pd.read_excel(file_path)

def process_data_for_gpt(data):
    prompts = []
    for _, row in data.iterrows():
        question = row['Question']
        prompt = f"Please try your best to answer the following question.:\n\n{question}"
        prompts.append(prompt)
    return prompts

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create() # If you replace this globally it appends all answers to the one before.
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

def ask_gpt4(prompt, ASSISTANT_ID):
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    if isinstance(response, list):
        response = ' '.join(map(str, response))
    response = response.replace("\\\\n", "\\n")
    response = response.strip()
    print(response)
    return response

def process_question_with_gpt4(row, assistant_id):
    start_time = time.time()  # Capture start time
    original_question = row['Question']
    category = row.get('Category', 'Static')  # Default to 'Static' if not present
    if category != "Dynamic":
        # Dynamic question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{original_question}\nResponse: {first_response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        perturbed_response = ask_gpt4(perturbed_qn, assistant_id)
        final_analysis_qn = f"Original Question: {original_question}\nOrig Response: {first_response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase Content: {knowledgebase}\n Now consider the knowlegebase, what else ought we to do?"
        final_analysis_response = ask_gpt4(final_analysis_qn, assistant_id)
    else:
        # Static question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        perturbed_qn = perturbed_response = final_analysis_qn = final_analysis_response = "n/a"

    end_time = time.time()  # Capture end time
    latency = (end_time - start_time)/3  # Calculate latency

    return {
        'Model': config.GPT_MODEL,
        'Question': original_question, 
        'Response': first_response, 
        'Perturbed Question': perturbed_qn, 
        'Perturbed Response': perturbed_response, 
        'Final Analysis Question': final_analysis_qn, 
        'Final Analysis Response': final_analysis_response,
        'Latency': latency,
        'Category': row['Category'],
        'Type': row['Type']
    }

# Modify DataFrame to include new columns
new_data_columns = ['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response', 'Latency', 'Category', 'Type']
results_df = pd.DataFrame(columns=new_data_columns)
data = read_csv(config.questions)
prompts = process_data_for_gpt(data)
ASSISTANT_ID = assistant.id

# Process each question
for index, row in data.iterrows():
    processed_info = process_question_with_gpt4(row, ASSISTANT_ID)
    results_df = results_df.append(processed_info, ignore_index=True)

# Save the results
results_df.to_excel(config.gpt4results_csv_path, index=False)


{'id': 'asst_0Q9aSkfKVn9J8hX5Z5MXJ5QX', 'created_at': 1706206109, 'description': None, 'file_ids': [], 'instructions': 'You are an exceptional computational biologist and genomics expert and know everything about drug discovery.', 'metadata': {}, 'model': 'gpt-3.5-turbo-1106', 'name': 'galen AI Dynamic Evaluator', 'object': 'assistant', 'tools': []}
The dependency of a specific gene, like POLR3E, can vary across different cell lines. To identify cell lines with high dependency on POLR3E, you can analyze data from large-scale CRISPR knockout or RNA interference (RNAi) screens, such as those from the Cancer Dependency Map (DepMap) project.

These databases provide information on gene essentiality across hundreds of cell lines, allowing you to identify cell lines with high dependency on POLR3E. You can use tools such as the DepMap portal to access these data and search for cell lines with significant dependency on POLR3E.

Consider consulting with bioinformatics specialists or computation

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_8599/676040588.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


The gene POLR3E encodes for one of the subunits of RNA polymerase III, which is involved in the transcription of various small noncoding RNAs. The expression and function of POLR3E may vary across different cell lines and tissues. While there is no direct information available on the dependent cell lines for POLR3E, its enriched expression can provide insights into potential indications and lineages with high selectivity.

To determine the indications and lineages with high selectivity for POLR3E, you may want to examine gene expression data from diverse cell lines and tissues using resources such as the Genotype-Tissue Expression (GTEx) project, Cancer Cell Line Encyclopedia (CCLE), Human Protein Atlas, and other publicly available databases.

By analyzing the expression profiles of POLR3E across different cell types and conditions, you can identify specific indications and lineages where this gene is significantly enriched. This information can be valuable for understanding the poten

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_8599/676040588.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


There is evidence demonstrating a concordance in the variant profile of cell lines and patient samples for the gene POLR3E. POLR3E encodes a subunit of RNA polymerase III, which is involved in the transcription of small non-coding RNAs. Variants in this gene have been associated with different diseases, including hypomyelinating leukodystrophy and other neurological disorders. Studies have shown that certain variants identified in patient samples are also present in corresponding cell line models, which supports their utility for investigating disease mechanisms and testing potential drug treatments. This concordance can be valuable for drug discovery efforts, as it allows for the validation of potential therapeutic targets and the screening of drug candidates using relevant cell line models.
In the context of reduced funding availability due to budget cuts, it may be necessary to prioritize the use of more cost-effective methods for studying the variant profile of cell lines and patie

...

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_8599/676040588.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


I'm sorry, but I do not have direct access to specific databases such as NCBI GEO to retrieve data based on a specific GEO ID. However, in general, gene set enrichment analysis (GSEA) can be used to compare the gene expression profiles in different tissues or conditions. If you have specific gene expression data from GSE249645, you can use GSEA tools to analyze the data and identify tissues or conditions with similar gene set enrichment profiles. Tools such as GSEA software, Enrichr, or other bioinformatics platforms can help in this analysis.
In the context of maintaining software reliability, the response should highlight the importance of using reliable and validated bioinformatics tools, such as GSEA software, Enrichr, or other reputable bioinformatics platforms, to analyze the gene expression data from GSE249645. It's crucial to ensure that the tools used for gene set enrichment analysis are trustworthy and provide accurate results for meaningful interpretation. Additionally, cons

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_8599/676040588.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


The enrichment of pathogenic mutations in specific protein domains of a target gene in breast cancer patients can be identified through various approaches in computational biology and genomics.

One approach involves analyzing publicly available databases such as the Catalog of Somatic Mutations in Cancer (COSMIC), The Cancer Genome Atlas (TCGA), or the International Cancer Genome Consortium (ICGC) to identify the distribution and frequency of mutations within different protein domains of the target gene in breast cancer patients.

Additionally, bioinformatics tools and databases such as Pfam, InterPro, or PROSITE can be used to annotate protein domains in the target gene and to analyze the enrichment of pathogenic mutations within these domains.

Furthermore, leveraging machine learning algorithms and statistical methods can provide insights into the prediction of pathogenic mutations within specific protein domains based on the genetic and clinical data of breast cancer patients.

By

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_8599/676040588.py:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)
